### Imports

In [1]:
import requests
from typing import List, Optional, Literal
from datetime import datetime
from llama_index.core.query_pipeline import QueryPipeline as QP
from llama_index.legacy.service_context import ServiceContext
from llama_index.core import VectorStoreIndex, load_index_from_storage
from sqlalchemy import text
from llama_index.core.schema import TextNode
from llama_index.core.storage import StorageContext
from pathlib import Path
from typing import Dict
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)
from llama_index.core.workflow import Workflow
from pyvis.network import Network
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core.prompts import PromptTemplate
from llama_index.core.query_pipeline import FnComponent
from llama_index.core.llms import ChatResponse
from llama_index.core.llms import ChatMessage, MessageRole
import pandas as pd

import json
import os

# put data into sqlite db
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex

from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)
from dotenv import load_dotenv

from llama_index.core.workflow import (
    step, 
    Context, 
    Workflow, 
    Event, 
    StartEvent, 
    StopEvent
)
#from llama_index.llms.anthropic import Anthropic
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.tools import FunctionTool
from enum import Enum
from typing import Optional, List, Callable, Tuple
from llama_index.utils.workflow import draw_all_possible_flows
from colorama import Fore, Back, Style
import asyncio
import nest_asyncio
from pydantic import BaseModel, Field
from llama_index.llms.openai import OpenAI as OpenAIIndex
import openai

In [2]:
load_dotenv()

client = openai.OpenAI()
MODEL = 'gpt-4o-2024-08-06'

# Get the environment variables
host = os.getenv('MYSQL_DB_HOST')
user = os.getenv('MYSQL_DB_USER')
password = os.getenv('MYSQL_DB_PASSWORD')
database = os.getenv('MYSQL_SALES_DB_NAME')

appsheet_app_id = os.getenv('APPSHEET_APP_ID')
appsheet_api_key = os.getenv('APPSHEET_API_KEY')

# Construct the connection string
connection_string = f"mysql+pymysql://{user}:{password}@{host}/{database}"

# Create the engine
engine = create_engine(connection_string)

metadata_obj = MetaData()
sql_database = SQLDatabase(engine)
table_node_mapping = SQLTableNodeMapping(sql_database)
sql_retriever = SQLRetriever(sql_database)

### CONTEXTO EN VIVO - Data indexing

In [3]:
def index_all_tables(
    sql_database: SQLDatabase, table_index_dir: str = "./table_indices"
) -> Dict[str, VectorStoreIndex]:
    """Index all tables."""

    table_names = ['CATEGORÍAS CAJA', 'CATEGORÍAS PRODUCTOS', 'CLIENTES', 'COLORES', 'ESTADOS', 'IVA', 'MÉTODOS DE PAGO', 'PERSONAL', 'PRODUCTOS', 'PROVEEDORES']
    # no indexd: CAJA, CHEQUES, PRODUCTOS PEDIDOS, STOCK, CUENTAS CORRIENTES, PEDIDOS, CONTROL DE PRECIOS

    if not Path(table_index_dir).exists():
        os.makedirs(table_index_dir)

    vector_index_dict = {}
    engine = sql_database.engine
    for table_name in table_names: #sql_database.get_usable_table_names():
        print(f"Indexing rows in table: {table_name}")

        if not os.path.exists(f"{table_index_dir}/{table_name}"):
            # get all rows from table
            with engine.connect() as conn:

                columns_query=(
                    f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}'"
                    f"AND TABLE_SCHEMA = '{database}'"
                )

                cursor = conn.execute(text(columns_query))
                result = cursor.fetchall()
                columns = []
                for column in result:
                    columns.append(column[0]) # get the first and only element of the tuple (the name)

                cursor = conn.execute(text(f'SELECT * FROM `{table_name}`'))
                result = cursor.fetchall()
                row_tups = []
                for row in result:
                    row_tups.append(tuple(row))
                    #print(dict(zip(columns, row)))

            # index each row, put into vector store index
            # TODO: CHECK THIS LINE: metadata
            nodes = [
                TextNode(text=str(t), 
                         metadata=dict(zip(columns, 
                                           #check rows types
                                           [str(value) if isinstance(value, datetime) else value 
                                            for value in t]
                                           ))) 
                for t in row_tups]

            # put into vector store index (use OpenAIEmbeddings by default)
            index = VectorStoreIndex(nodes) #service_context=service_context

            # save index
            index.set_index_id("vector_index")
            index.storage_context.persist(f"{table_index_dir}/{table_name}")
        else:
            print('index already exists')
            # rebuild storage context
            storage_context = StorageContext.from_defaults(
                persist_dir=f"{table_index_dir}/{table_name}"
            )
            # load index
            index = load_index_from_storage(
                storage_context, index_id="vector_index") #service_context=service_context
            
        vector_index_dict[table_name] = index

    return vector_index_dict

vector_index_dict = index_all_tables(sql_database)

Indexing rows in table: CATEGORÍAS CAJA
index already exists
Indexing rows in table: CATEGORÍAS PRODUCTOS
index already exists
Indexing rows in table: CLIENTES
index already exists
Indexing rows in table: COLORES
index already exists
Indexing rows in table: ESTADOS
index already exists
Indexing rows in table: IVA
index already exists
Indexing rows in table: MÉTODOS DE PAGO
index already exists
Indexing rows in table: PERSONAL
index already exists
Indexing rows in table: PRODUCTOS
index already exists
Indexing rows in table: PROVEEDORES
index already exists


#### Intermediate steps testing

In [4]:
test_retriever = vector_index_dict["CLIENTES"].as_retriever(
    similarity_top_k=1
)
nodes = test_retriever.retrieve("cliente")
print(nodes[0].get_content())
print(nodes[0].metadata)

(996778, 'Jorge Botta', 1112345689, 4513882, 'Oran 3196', None, None, None, None, None, None, None)
{'ID': 996778, 'CLIENTE': 'Jorge Botta', 'TELEFONO': 1112345689, 'DNI/CUIT': 4513882, 'DIRECCION': 'Oran 3196', 'LIMITE_DE_SALDO': None, 'CUENTA_CORRIENTE': None, 'RESTO_DE_SALDO': None, 'IVA': None, 'IVA_MINIMO': None, 'LISTA_DE_PRECIOS': None, 'USUARIO': None}


In [5]:
test_retriever = vector_index_dict["PRODUCTOS"].as_retriever(
    similarity_top_k=1
)
nodes = test_retriever.retrieve("RM X 30 KG LINEA NORT PREMIUM") #RM X 30 KG LINEA NORT PREMIUM
#print(nodes[0].get_content(metadata_mode='all'))
print(nodes[0].metadata)
#response.source_nodes[0].node.metadata['file_name']


{'ID': 1, 'PRODUCTO': 'RM X 30 KG LINEA NORT PREMIUM', 'CATEGORÍA': 'REVESTIMIENTO', 'PRECIO ESTANDAR S/IVA': 41400.0, 'PRECIO INTENSO S/IVA': 47035.0, 'IVA': '21', 'STOCK FÁBRICA': 284, 'VALOR STOCK': 11757600.0, 'VALOR STOCK TOTAL': 'VALOR TOTAL'}


### PROMPTEO - Classes definition

In [6]:
class InitializeEvent(Event):
    pass

class ConciergeEvent(Event):
    request: Optional[str] = None
    just_completed: Optional[str] = None
    need_help: Optional[bool] = None

class OrchestratorEvent(Event):
    request: str

class OrderCreationEvent(Event):
    request: str

class OrderUpdateEvent(Event):
    request: str

class StockManagerEvent(Event):
    request: str


chat_history = []

In [7]:
class ConciergeAgent():
    name: str
    parent: Workflow
    tools: list[FunctionTool]
    system_prompt: str
    context: Context
    current_event: Event
    trigger_event: Event

    def __init__(
            self,
            parent: Workflow,
            tools: List[Callable], 
            system_prompt: str, 
            trigger_event: Event,
            context: Context,
            name: str,
        ):
        self.name = name
        self.parent = parent
        self.context = context
        self.system_prompt = system_prompt
        self.context.data["redirecting"] = False
        self.trigger_event = trigger_event

        def explain_steps(steps:str) -> None:
            """Explain what you'll do with the request, step by step."""
            print(Fore.RED + "[explain_steps] is being executed" + Style.RESET_ALL)
            print(f"{self.name} will explain steps:", steps)
            context.session.send_event(StopEvent())

        def done() -> None:
            """When you complete your task, call this tool."""
            print(Fore.RED + "[done] is being executed" + Style.RESET_ALL)
            print(f"{self.name} is complete")
            self.context.data["redirecting"] = True
            context.session.send_event(ConciergeEvent(just_completed=self.name))

        def need_help() -> None:
            """If the user asks to do something you don't know how to do, call this."""
            print(Fore.RED + "[need_help] is being executed" + Style.RESET_ALL)
            print(f"{self.name} needs help")
            self.context.data["redirecting"] = True
            context.session.send_event(StopEvent())

        self.tools = [
            FunctionTool.from_defaults(fn=done),
            FunctionTool.from_defaults(fn=need_help),
            FunctionTool.from_defaults(fn=explain_steps),
        ]
        for t in tools:
            self.tools.append(FunctionTool.from_defaults(fn=t))

        agent_worker = FunctionCallingAgentWorker.from_tools(
            self.tools,
            llm=self.context.data["llm"],
            allow_parallel_tool_calls=False,
            system_prompt=self.system_prompt,
            verbose=True,
        )
        self.agent = agent_worker.as_agent()        

    async def handle_event(self, ev: Event):
        print(Fore.RED + "[handle_event] is being executed" + Style.RESET_ALL)
        self.current_event = ev

        response = str(self.agent.chat(ev.request, chat_history=chat_history))
        chat_history.append(ChatMessage(role=MessageRole.ASSISTANT, content=str(response)))
        print("agent printing!", self.name)
        print(Fore.MAGENTA + str(response) + Style.RESET_ALL)

        # send message to user using whatsapp api
        #await send_message_to_user(message=str(response), to='5491131500591')

        # if they're sending us elsewhere we're done here
        if self.context.data["redirecting"]:
            self.context.data["redirecting"] = False
            return None

        # otherwise, get some user input and then loop
        user_msg_str = input("> ").strip()
        chat_history.append(ChatMessage(role=MessageRole.USER, content=user_msg_str))
        if user_msg_str == "":
            return StopEvent()
        return self.trigger_event(request=user_msg_str)


In [8]:
class Product(BaseModel):
    ID_PRODUCTO: int = Field(..., strict=True, title="Product ID found in the database")
    TIPO: Literal['ESTANDAR', 'INTENSO', 'PIEDRA GRANITO'] = Field(..., title="Type of the color, each color has a corresponding type in the database")
    COLOR: str = Field(..., strict=True, title="Color of the product (TYPE GOES IN THE OTHER FIELD), in the database it is a foreign key")
    CANTIDAD: int = Field(..., strict=True, title="Quantity")

class Order(BaseModel):
    ID_CLIENTE: int = Field(..., strict=True, title="Customer ID found in the database")
    TIPO_DE_ENTREGA: Literal['CLIENTE', 'RETIRA EN FÁBRICA', 'OTRO']
    DIRECCION: Optional[str] = Field(..., strict=True, title="Delivery Address, if deliver_type is CLIENTE or RETIRA EN FABRICA is not required, else it is")
    METODO_DE_PAGO: Literal["EFECTIVO", 'DÓLARES', 'MERCADO PAGO', 'CHEQUE', 'TRANSFERENCIA BANCARIA']
    NOTA: Optional[str] = Field(None, strict=True, title="Note only used if explicitly specified")
    # products: List[Product]

class OrderAndProductList(BaseModel):
    order: Order
    product_list: List[Product]

def appsheet_add(rows: List[Dict] | Dict, table_name: str):
    print(Fore.RED + "[appsheet_add] is being executed" + Style.RESET_ALL)

    if isinstance(rows, dict):
        rows = [rows]

    products_url = f"https://api.appsheet.com/api/v2/apps/{appsheet_app_id}/tables/{table_name}/Action"

    headers = {
        "Content-Type": "application/json",
        "ApplicationAccessKey": appsheet_api_key
    }

    request = {
        "Action": "Add",
        "Properties": {"Locale": "en-US"},
        "Rows": rows
    }

    response = requests.post(products_url, headers=headers, json=request)
    return response

def appsheet_edit(rows: List[Dict] | Dict, table_name: str):
    print(Fore.RED + "[appsheet_edit] is being executed" + Style.RESET_ALL)

    if isinstance(rows, dict):
        rows = [rows]

    products_url = f"https://api.appsheet.com/api/v2/apps/{appsheet_app_id}/tables/{table_name}/Action"

    headers = {
        "Content-Type": "application/json",
        "ApplicationAccessKey": appsheet_api_key
    }

    request = {
        "Action": "Edit",
        "Properties": {"Locale": "en-US"},
        "Rows": rows
    }

    print(request)

    response = requests.post(products_url, headers=headers, json=request)
    return response


In [29]:
class OrderWorkflow(Workflow):
            
    @step(pass_context=True)
    async def initialize(self, ctx: Context, ev: InitializeEvent) -> ConciergeEvent:

        ctx.data["user"] = {
            "username": None,
            #"session_token": None,
            #"account_id": None,
            #"account_balance": None,
        }
        ctx.data["success"] = None
        ctx.data["redirecting"] = None
        ctx.data["overall_request"] = None
        ctx.data["order"] = {}
        ctx.data["order"]["products"] = []

        # Print the contents of ctx.data for debugging
        print("ctx.data:", ctx.data)

        ctx.data["llm"] = OpenAIIndex(model="gpt-4o-mini",temperature=0)
        #ctx.data["llm"] = Anthropic(model="claude-3-5-sonnet-20240620",temperature=0.4)
        #ctx.data["llm"] = Anthropic(model="claude-3-opus-20240229",temperature=0.4)
        return ConciergeEvent()
  
    @step(pass_context=True)
    async def concierge(self, ctx: Context, ev: ConciergeEvent | StartEvent) -> InitializeEvent | StopEvent | OrchestratorEvent:
        
        response = None
        
        # initialize user if not already done
        if ("user" not in ctx.data):
            return InitializeEvent()
        
        # initialize concierge if not already done
        if ("concierge" not in ctx.data):
            system_prompt = (f"""
                Usa el idioma español (argentina).             
                Sos un asistente útil que ayuda a un empleado a manejar el software de su empresa, usa el idioma español (argentina).
                Tu trabajo es hacerle preguntas al empleado para entender qué quiere hacer y brindarle las acciones disponibles que puede hacer.
                Eso incluye 
                             * crear un nuevo pedido de productos a un cliente en el sistema
                             * crear un nuevo cliente 
                Cuando el usuario termine con su primer tarea, recuerdale el resto de tareas que podés hacer
                             """)

            agent_worker = FunctionCallingAgentWorker.from_tools(
                tools=[],
                llm=ctx.data["llm"],
                allow_parallel_tool_calls=False,
                system_prompt=system_prompt
            )
            ctx.data["concierge"] = agent_worker.as_agent()        

        concierge = ctx.data["concierge"]
        if ctx.data["overall_request"] is not None:
            print("There's an overall request in progress, it's ", ctx.data["overall_request"])
            last_request = ctx.data["overall_request"]
            ctx.data["overall_request"] = None
            return OrchestratorEvent(request=last_request)
        elif (ev.just_completed is not None):
            response = concierge.chat(f"FYI, the user has just completed the task: {ev.just_completed}")
            chat_history.append(ChatMessage(role=MessageRole.ASSISTANT, content=str(response)))
        elif (ev.need_help):
            print("The previous process needs help with ", ev.request)
            return OrchestratorEvent(request=ev.request)
        elif (ev.request is not None):
            response = concierge.chat(ev.request)
            chat_history.append(ChatMessage(role=MessageRole.ASSISTANT, content=str(response)))
        

        if response is not None: 
            print("concierge!!!!")
            print(Fore.MAGENTA + str(response) + Style.RESET_ALL)

        # concierge send message to user using whatsapp api 
        #await send_message_to_user(message=str(response), to='5491131500591')

        user_msg_str = input("> ").strip()
        chat_history.append(ChatMessage(role=MessageRole.USER, content=user_msg_str))
        if user_msg_str == "":
            print("User has stopped the system with ''")
            return StopEvent()
        #user_msg_str = await user_input.get()
        return OrchestratorEvent(request=user_msg_str)
    
    @step(pass_context=True)
    async def orchestrator(self, ctx: Context, ev: OrchestratorEvent) -> ConciergeEvent | OrderCreationEvent | StopEvent | StockManagerEvent:

        #print(f"Orchestrator received request: {ev.request}")

        def emit_order_management() -> bool:
            """Call this if the user wants create or manage an order in the system."""      
            print("__emitted: order creation")      
            ctx.session.send_event(OrderCreationEvent(request=ev.request))
            return True
        
        def emit_stock_manager() -> bool:
            """Call this if the user wants to create a new stock movement in the system."""
            print("__emitted: stock manager")
            ctx.session.send_event(StockManagerEvent(request=ev.request))
            return True
        
        """ def emit_update_order() -> bool:
            Call this if the user wants create a new order in the system
            print("__emitted: order creation")      
            ctx.session.send_event(OrderUpdateEvent(request=ev.request))
            return True """

        def emit_concierge() -> bool:
            """Call this if the user wants to do something else or you can't figure out what they want to do."""
            print("__emitted: concierge (non stop)")
            #ctx.session.send_event(ConciergeEvent(request=("This is the request, the orchestator didn't know how to continue: "+ev.request)))
            #ctx.session.send_event(StopEvent())
            self.send_event(ConciergeEvent(request=ev.request))
            return True

        def emit_stop() -> bool:
            """Call this if the user wants to stop or exit the system."""
            print("__emitted: stop")
            ctx.session.send_event(StopEvent())
            return True
        
        # define tool for getting customer data
        def getCustomerData(customer_name: str) -> str:
            """Returns the customer metadata from the customer name given by the user to send that value to the system later"""
            print("__emitted: getCustomerData")
            test_retriever = vector_index_dict["CLIENTES"].as_retriever(similarity_top_k=1)
            nodes =  test_retriever.retrieve(customer_name)
            if nodes[0].metadata is None:
                print("No customer found")
                ctx.session.send_event(StopEvent())
            print(f"For customer {customer_name} found:", 
                  nodes[0].metadata["CLIENTE"], nodes[0].metadata["ID"])
            return str(nodes[0].metadata)

        tools = [
            FunctionTool.from_defaults(fn=emit_order_management),
            FunctionTool.from_defaults(fn=emit_concierge),
            FunctionTool.from_defaults(fn=emit_stop),
            FunctionTool.from_defaults(fn=getCustomerData),
            FunctionTool.from_defaults(fn=emit_stock_manager),
            #FunctionTool.from_defaults(fn=emit_update_order)
        ]
        
        system_prompt = (f"""
            You are on orchestration agent.
            Your job is to decide which agent to run based on the current state of the user and what they've asked to do. 
            You run an agent by calling the appropriate tool for that agent.
            in your response, only select the tool, don't add any text into it.      
            If there's no clear use for one of the tools, call the tool "concierge" to signal that the concierge agent should help, 
            UNLESS THERE'S AN ERROR, YOU ALWAYS HAVE TO CALL ONE OF THE TOOLS, if you don't call any tool, the system will break.
            If you NOTICE SOMETHING IS NOT WORKING or did not call any tools, return the string "FAILED" followed by the exact reason you are selecting this option.
        """)

        """when you call the concierge, also send information to help the concierge answer the user request, as you have more information than him 
                         (your reply will directly sent to the concierge, the user won't read it)."""
        
        agent_worker = FunctionCallingAgentWorker.from_tools(
            tools=tools,
            llm=ctx.data["llm"],
            allow_parallel_tool_calls=False,
            system_prompt=system_prompt,
            verbose=True
        )
        ctx.data["orchestrator"] = agent_worker.as_agent()        
        
        orchestrator = ctx.data["orchestrator"]
        response = str(orchestrator.chat(ev.request))
        print(Fore.MAGENTA + "Orchestrator: " +response + Style.RESET_ALL)
        chat_history.append(ChatMessage(role=MessageRole.ASSISTANT, content=("Información del orquestador: "+str(response))))

        if "FAILED" in response:
            print(Fore.RED + response + Style.RESET_ALL)
            return StopEvent()
            #return OrchestratorEvent(request=ev.request)
    
    @step(pass_context=True)
    async def order_manager(self, ctx: Context, ev: OrderCreationEvent) -> ConciergeEvent | StopEvent:

        if("order_manager_agent" not in ctx.data):

            def send_order():
                """Useful for sending the order to the system via API request, Order will be stored in the session context. 
                Make sure to call the other tools to get the IDs before, and that you have all the data needed."""

                prompt = """
                You will be provided with data about an order, but you only have to list its products to inser in a database.
                Your goal will be to parse the data following the schema provided.
                Here is a description of the parameters:
                - product: specifies product_id, product type, color, and quantity
                - order: general data about the order, and it has a list of products that the user will list
                Tell the user the order number id when it's succesfully created

                """
                
                response = client.beta.chat.completions.parse(
                    model=MODEL,
                    temperature=0,
                    messages=[
                        {"role": "system", "content": prompt},
                        {"role": "user", "content": str(ctx.data["order"])}
                    ],
                    response_format=OrderAndProductList
                )
                print(json.loads(response.choices[0].message.content))

                products = json.loads(response.choices[0].message.content)["product_list"]
                order = json.loads(response.choices[0].message.content)["order"]

                response = appsheet_add([order], "PEDIDOS")

                if isinstance(response, requests.Response) and response.status_code != 200:
                    print(f"Error inserting order: {response.text}")
                    return f"Error inserting order, stop the system"
                elif response.status_code == 200 and response.json().get('Rows'):

                    print(f"Order inserted successfully, inserting products now", response.json().get('Rows'))

                    order = response.json().get('Rows')[0]

                    for product in products:
                        product["ID_PEDIDO"] = order["ID_KEY"]

                    response = appsheet_add(products, "PRODUCTOS PEDIDOS")

                    if isinstance(response, requests.Response) and response.status_code != 200:
                        print(f"Error inserting order: {response.text}")
                        return f"Error inserting products, stop the system"
                    elif response.status_code == 200 and response.json().get('Rows'):

                        response = appsheet_edit({"ID_KEY": order["ID_KEY"], "GUARDADO": 1}, "PEDIDOS")
                        if isinstance(response, requests.Response) and response.status_code == 200:
                            print(f"Products. Order inserted successfully , Order ID: {order['ID_KEY']}", response.json().get('Rows'))
                            return f"Order and products created successfully, Order ID: {order['ID_KEY']}"

                        
                        else:
                            print(f"Products. Error in the request body or url, check the settings")
                            return f"Error in the request body or url, check the settings, stop the system"
                    else:
                        print(f"Products. Error in the request body or url, check the settings")
                        return f"Error in the request body or url, check the settings, stop the system"
      
                else:
                    print(f"Order. Error in the request body or url, check the settings")
                    return f"Error in the request body or url, check the settings, stop the system"
            
            def getCustomerID(customer_name: str) -> str:
                """Returns the customer ID from the customer name given by the user to send that value to the system later"""
                test_retriever = vector_index_dict["CLIENTES"].as_retriever(similarity_top_k=3)
                nodes = test_retriever.retrieve(customer_name)

                prompt = f"""
                You are part of a complex system that relies on you to get the correct customer ID.
                You are given a customer name and a list of possible matches. 
                Your task is to select the best match based on the name.
                Only return the ID of the best match with no quotes, nothing else.
                Customer Name: {customer_name}
                Matches: {[node.metadata for node in nodes]}
                """
                response = ctx.data["llm"].chat(prompt)
                customer_id = response.choices[0].message.content.strip()

                # Check if the customer ID matches any of the metadata
                for node in nodes:
                    if node.metadata["ID"] == customer_id:
                        return customer_id

                print("No matching customer found")
                ctx.session.send_event(StopEvent())
                return "Error: No matching customer found"
            
            def createOrder(customer_name:str, payment_method:str, shipment_type_and_info:str, shipment_adress:Optional[str] = "", nota:Optional[str] = "") -> str:
                """Creates an order in the session, it will be used to store the order data before sending it to the system. 
                Don't use it before having the needed data"""

                customer_id = getCustomerID(customer_name)
                if customer_id == "Error: No matching customer found, ask the user for more details about the customer name": 
                    return customer_id

                ctx.data["order"]["ID_CLIENTE"] = customer_id
                ctx.data["order"]["TIPO_DE_ENTREGA"] = shipment_type_and_info
                ctx.data["order"]["DIRECCION"] = shipment_adress
                ctx.data["order"]["METODO_DE_PAGO"] = payment_method
                ctx.data["order"]["NOTA"] = nota
                print("Order created:",  str(ctx.data["order"]))

                return "Order created successfully, make sure to add products before sending it to the system"
            
            #DNI/CUIT
            def getCustomerCUIT(customer_name: str) -> str:
                """Returns the CUIT of the customer from the customer name given by the user to send that value to the system later"""
                test_retriever = vector_index_dict["CLIENTES"].as_retriever(similarity_top_k=1)
                nodes =  test_retriever.retrieve(customer_name)
                print(f"For customer {customer_name} found:", 
                      nodes[0].metadata["CLIENTE"]), nodes[0].metadata["DNI/CUIT"]
                if nodes[0].metadata is None:
                    print("No customer found")
                    ctx.session.send_event(StopEvent())
                return nodes[0].metadata["DNI/CUIT"]
            
            def addProductToOrder(product_name:str, color_description:str, quantity:int):
                """Adds a product to the order in the current session, product_name is the name of the product (not the color), color description (with its type if the user provided it), and quantity.
                Make sure to add all the products using this tool."""
                
                color_tuple = getColorName(color_description)

                if color_tuple == "Error: No matching color found":
                    return "Error: No matching color found, stop the system, ask the user for more details about the color"
                
                product_id = getProductID(product_name)
                if product_id == "Error: No matching product found":
                    return "Error: No matching product found, stop the system, ask the user for more details about the product"

                product = {
                    "CANTIDAD": quantity, 
                    "ID_PRODUCTO": product_id, 
                    "COLOR": color_tuple[0], 
                    "TIPO": color_tuple[1]
                    }
                ctx.data["order"]["products"].append(product)
                print(f"Product added to the order: {product}")

                return "Product added to the order, make sure that all the products are added, and to load other order info too. Don't forget to send it to the system when all the data is loaded"
            
            def getProductID(product_description_name: str) -> str:
                """Returns the product ID from the product name given by the user to send that value to the system later"""
                test_retriever = vector_index_dict["PRODUCTOS"].as_retriever(similarity_top_k=3)
                nodes = test_retriever.retrieve(product_description_name)

                prompt = f"""
                You are part of a complex system that relies on you to get the correct product ID.
                You are given a product description and a list of possible matches. 
                Your task is to select the best match based on the description.
                Only return the ID of the best match with no quotes, nothing else.
                Product Description: {product_description_name}
                Matches: {[node.metadata for node in nodes]}
                """
                response = ctx.data["llm"].chat(prompt)
                product_id = response.choices[0].message.content.strip()

                # Check if the product ID matches any of the metadata
                for node in nodes:
                    if node.metadata["ID"] == product_id:
                        return product_id

                print("No matching product found")
                ctx.session.send_event(StopEvent())
                return "Error: No matching product found"
            
            def getColorName(color_description: str) -> Tuple[str, str]:
                """Returns the top 3 search for the Color exact name with its corresponding type from the color given by the user to send that value to the system later,
                based on the color description given by the user, choose the correct one"""
                test_retriever = vector_index_dict["COLORES"].as_retriever(similarity_top_k=3)
                nodes = test_retriever.retrieve(color_description)

                prompt = f"""
                You are part of a complex system that relies on you to get the correct color and type for the products.
                You are given a description of a color and a list of possible matches. 
                Your task is to select the best match based on the description.
                Only return the COLOR and TIPO of the best match with the format "COLOR, TIPO" with no quotes, nothing else.
                Description: {color_description}
                Matches: {[node.metadata for node in nodes]}
                """
                response = ctx.data["llm"].chat(prompt)
                color_info = response.choices[0].message.content.strip().split(", ")

                # Check if the color info matches any of the metadata
                for node in nodes:
                    if node.metadata["COLOR"] == color_info[0] and node.metadata["TIPO"] == color_info[1]:
                        return color_info[0], color_info[1]

                print("No matching color found")
                ctx.session.send_event(StopEvent())
                return "Error: No matching color found"
        
            def udpate_order(order_id:int, new_status:str) -> str:
                """Updates the status of an order in the system, given its ID (as int) (or sometimes referenced as number of order, pedido o remito) and the new status
                Available statuses are: 'ANULADO' 'En expreso' 'En viaje' 'Listo para despachar' 'Nuevo pedido' 'Pedido tomado' 'Pidiendo' 'Recibido'"""

                if new_status not in ['ANULADO', 'En expreso', 'En viaje', 'Listo para despachar', 'Nuevo pedido', 'Pedido tomado', 'Pidiendo', 'Recibido']:
                    return "Invalid status, check the available statuses"
                
                # get all rows from table
                with engine.connect() as conn:

                    columns_query=(
                        f"SELECT ID_KEY FROM PEDIDOS WHERE ID = '{order_id}'"
                    )

                    cursor = conn.execute(text(columns_query))
                    result = cursor.fetchall()
                    if len(result) != 1:
                        print(f"Error: Expected one row, but got {len(result)} rows")
                        return f"Error: Expected one row, but got {len(result)} rows"
                    
                    id_key = result[0][0]

                products_url = f"https://api.appsheet.com/api/v2/apps/{appsheet_app_id}/tables/PEDIDOS/Action"

                headers = {
                    "Content-Type": "application/json",
                    "ApplicationAccessKey": appsheet_api_key
                }

                request = {
                    "Action": "Edit",
                    "Properties": {"Locale": "en-US"},
                    "Rows": [
                        {
                            "ID_KEY": id_key,
                            "ESTADO": new_status
                        }
                    ]
                }

                print(request)

                response = requests.post(products_url, headers=headers, json=request)

                if isinstance(response, requests.Response) and response.status_code != 200:
                    print(f"Error updateing order: {response.text}")
                    return f"Error updating order, stop the system"
                elif response.status_code == 200 and response.json().get('Rows'):

                    print(f"Order updated successfully")
                    return "Order updated successfully"
                else:
                    print(f"Order. Error in the request body or url, check the settings")
                    return f"Error in the request body or url, check the settings, stop the system"
                            
                
            
            def stop() -> None:
                """Call this if you notice that something is not working propperly."""
                print("Order creator agent is stopping")
                ctx.session.send_event(StopEvent())     
            
            system_prompt = (f"""
                
                Usa el idioma español (argentina).
                You are a helpful assistant that recollects data to create an order correctly in the system by sending an API request.
                This is the ORDER structure:
                            {str(Product)}
                            {str(Order)}
                
                You should use the "createOrder" and "addProductToOrder" for all the products to correctly create the order. 
                    
                Once you have loaded all the data, call the "send_order" tool and it will send the order to the system taken the data you have just loaded.
                if you don't send the order, you'll break the system
                """)
             
            ctx.data["order_manager_agent"] = ConciergeAgent(
                name="Order Manager Agent",
                parent=self,
                tools=[send_order, stop, addProductToOrder, createOrder, getCustomerCUIT, getProductID, getColorName, udpate_order],
                context=ctx,
                system_prompt=system_prompt,
                trigger_event=OrderCreationEvent
            )

        return await ctx.data["order_manager_agent"].handle_event(ev)

    @step(pass_context=True)
    async def stock_manager(self, ctx: Context, ev: StockManagerEvent) -> ConciergeEvent | StopEvent:
        if "stock_manager_agent" not in ctx.data:
            # Define function to create a stock movement
            def create_stock_movement(product_description: str, quantity: int) -> str:
                """Creates a stock movement in the STOCK table."""
                
                # Retrieve the product ID based on the product description
                test_retriever = vector_index_dict["PRODUCTOS"].as_retriever(similarity_top_k=3)
                nodes = test_retriever.retrieve(product_description)

                prompt = f"""
                You are part of a complex system that relies on you to get the correct product ID.
                You are given a product description and a list of possible matches. 
                Your task is to select the best match based on the description.
                Only return the ID of the best match with no quotes, nothing else.
                Product Description: {product_description}
                Matches: {[node.metadata for node in nodes]}
                """
                response = ctx.data["llm"].chat(prompt)
                product_id = response.choices[0].message.content.strip()

                # Check if the product ID matches any of the metadata
                for node in nodes:
                    if node.metadata["ID"] == product_id:
                        stock_movement = {
                            "ID PRODUCTO": product_id,
                            "CANTIDAD": quantity
                        }
                        response = appsheet_add([stock_movement], "STOCK")
                        if response.status_code == 200:
                            return "Stock movement created successfully."
                        else:
                            return f"Error creating stock movement: {response.text}"

                print("No matching product found")
                ctx.session.send_event(StopEvent())
                return "Error: No matching product found, ask the user for more details about the product"
            
            def getProductID(product_description_name: str) -> str:
                """Returns the product ID from the product name given by the user to send that value to the system later"""
                test_retriever = vector_index_dict["PRODUCTOS"].as_retriever(similarity_top_k=3)
                nodes = test_retriever.retrieve(product_description_name)

                prompt = f"""
                You are part of a complex system that relies on you to get the correct product ID.
                You are given a product description and a list of possible matches. 
                Your task is to select the best match based on the description.
                Only return the ID of the best match with no quotes, nothing else.
                Product Description: {product_description_name}
                Matches: {[node.metadata for node in nodes]}
                """
                response = ctx.data["llm"].chat(prompt)
                product_id = response.choices[0].message.content.strip()

                # Check if the product ID matches any of the metadata
                for node in nodes:
                    if node.metadata["ID"] == product_id:
                        return product_id

                print("No matching product found")
                ctx.session.send_event(StopEvent())
                return "Error: No matching product found"
            
            
            def stop() -> None:
                """Call this if you notice that something is not working propperly."""
                print("Stock manager agent is stopping")
                ctx.session.send_event(StopEvent())

            # Define tools for the agent
            tools = [
                FunctionTool.from_defaults(fn=create_stock_movement),
                FunctionTool.from_defaults(fn=stop),
            ]

            # Define system prompt for the agent
            system_prompt = "You are a stock manager agent responsible for creating stock movements in the system."

            # Initialize the agent
            agent_worker = FunctionCallingAgentWorker.from_tools(
                tools=tools,
                llm=ctx.data["llm"],
                allow_parallel_tool_calls=False,
                system_prompt=system_prompt,
                verbose=True
            )
            ctx.data["stock_manager_agent"] = agent_worker.as_agent()

        # Get the agent from context
        stock_manager_agent = ctx.data["stock_manager_agent"]
        response = stock_manager_agent.chat(ev.request)
        print(Fore.MAGENTA + str(response) + Style.RESET_ALL)
        chat_history.append(ChatMessage(role=MessageRole.ASSISTANT, content=str(response)))

        # Get user input and continue or stop
        user_msg_str = input("> ").strip()
        if user_msg_str == "":
            return StopEvent()
        return StockManagerEvent(request=user_msg_str)


#draw_all_possible_flows(OrderWorkflow,filename="order_workflow.html")

In [10]:
chat_history

[]

In [11]:
#draw_all_possible_flows(OrderWorkflow,filename="order_workflow.html")

hola, quisiera agregar un pedido de 12 'RM de 30  LINEA NORT' color eucalipto estandar,  4 FONDO BASE de 20 linea nort color neutro estandar para jorge que va a retirar en fábrica y pagar en efectivo
 

hola, quisiera agregar un pedido de 12 'RM de 30  LINEA NORT' color eucalipto estandar para jorge que va a retirar en fábrica y pagar en efectivo

for every function, add a log printing with Fore.RED saying '[function_name] is being executed' so its easy to understand the workflow

agrega 23 al stock de rm de 30 linea nort

In [30]:
c = OrderWorkflow(timeout=500, verbose=True)
result = await c.run()
print(result)

Running step concierge
Step concierge produced event InitializeEvent
Running step initialize
ctx.data: {'user': {'username': None}, 'success': None, 'redirecting': None, 'overall_request': None, 'order': {'products': []}}
Step initialize produced event ConciergeEvent
Running step concierge


Step concierge produced event OrchestratorEvent
Running step orchestrator
Added user message to memory: agrega 23 al stock de rm de 30 linea nort
=== Calling Function ===
Calling function: emit_stock_manager with args: {}
__emitted: stock manager
=== Function Output ===
True
Orchestrator: 
Step orchestrator produced no event
Running step stock_manager
Added user message to memory: agrega 23 al stock de rm de 30 linea nort
=== Calling Function ===
Calling function: create_stock_movement with args: {"product_description": "rm de 30 linea nort", "quantity": 23}
=== Function Output ===
Encountered error: 1 validation error for LLMChatStartEvent
messages
  Input should be a valid list [type=list_type, input_value="\n                You ar...AL'}]\n                ", input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/list_type
=== Calling Function ===
Calling function: create_stock_movement with args: {"product_description": "rm de 30 linea nort", "quantity":

In [ ]:
""" import gradio as gr
import os
from openai import OpenAI
import assemblyai as aai
from tempfile import NamedTemporaryFile
from dotenv import load_dotenv

# Load API keys from environment variables
load_dotenv()
aai.settings.api_key = os.getenv("ASSEMBLYAI_API_KEY")
transcriber = aai.Transcriber()

# Test OpenAI TTS
def test_tts(test_text):
    #test_text = "Saucotec is gonna take over the world."
    speech_file_path = "test.mp3"  # NamedTemporaryFile(suffix=".mp3", delete=False).name

    try:
        response = client.audio.speech.create(
            model="tts-1",  # Make sure you have access to the TTS model
            voice="onyx",  # Ensure the voice exists
            input=test_text
        )
        response.stream_to_file(speech_file_path)
        #print(f"Audio saved to {speech_file_path}")
        return response
    except Exception as e:
        print(f"Error generating audio: {e}")

# Test AssemblyAI transcription
async def transcribe(audio_file_path):
    config = aai.TranscriptionConfig(speaker_labels=True, language_code="es")

    try:
        transcript = transcriber.transcribe(audio_file_path, config)

        if transcript.status == aai.TranscriptStatus.completed:
            print(f"Transcription: {transcript.text}")
            result = await c.run(
            message= transcript.text
            #"hola, quisiera agregar un pedido de 12 'RM de 30  LINEA NORT' color eucalipto estandar para jorge que va a retirar en fábrica y pagar en efectivo"
            #transcript.text
            )
            print("Finalizado:", main_response)
            test_tts(main_response)
            print("Audio generado")
            with open("test.mp3", "rb") as audio_file:
                audio_data = audio_file.read()
            return audio_data
        
    except Exception as e:
        print(f"Error transcribing audio: {e}")

# Create the Gradio interface
# Removed the 'source' parameter from gr.Audio
ui = gr.Interface(fn=transcribe, inputs=gr.Audio(type="filepath"), outputs="audio", live=True)

# Launch the interface and share it publicly
ui.launch(share=True)
 """

' import gradio as gr\nimport os\nfrom openai import OpenAI\nimport assemblyai as aai\nfrom tempfile import NamedTemporaryFile\nfrom dotenv import load_dotenv\n\n# Load API keys from environment variables\nload_dotenv()\naai.settings.api_key = os.getenv("ASSEMBLYAI_API_KEY")\ntranscriber = aai.Transcriber()\n\n# Test OpenAI TTS\ndef test_tts(test_text):\n    #test_text = "Saucotec is gonna take over the world."\n    speech_file_path = "test.mp3"  # NamedTemporaryFile(suffix=".mp3", delete=False).name\n\n    try:\n        response = client.audio.speech.create(\n            model="tts-1",  # Make sure you have access to the TTS model\n            voice="onyx",  # Ensure the voice exists\n            input=test_text\n        )\n        response.stream_to_file(speech_file_path)\n        #print(f"Audio saved to {speech_file_path}")\n        return response\n    except Exception as e:\n        print(f"Error generating audio: {e}")\n\n# Test AssemblyAI transcription\nasync def transcribe(audi